In [1]:
import requests as rq
import pandas as pd
import json
import bs4
import re
import math

In [2]:
with open("C:/open_data_key.txt", 'r', encoding='utf-8') as f:
    ServiceKey = f.readlines()[2]

In [3]:
def connectionXML(url,params):
    response = rq.get(url,params)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
#     soup = bs4.BeautifulSoup(xml, ['lxml', 'xml'])
    return soup

In [4]:
def NoneXMLdata(params):
    if params == None: return ''
    else: return params.text.strip()

In [5]:
# 특수문자 리스트
special_word = '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'

In [6]:
data_dic = {
         'buryu' : []
        ,'sort_cd' : []
        ,'pum_cd' : []
        ,'pum_nm' : []
        ,'corp_cd_1' : []
        ,'corp_cd_2' : []
        ,'corp_cd_3' : []
        ,'corp_cd_4' : []
        ,'corp_cd_5' : []
        ,'corp_cd_6' : []
        ,'sum_tot' : []
        ,'date' : []
    }

In [7]:
def getDataXML(result_xml):
    if len(data.find_all('list')) == 0:
        return
    # 데이터를 담을 딕셔너리
    
    result_list = result_xml.find_all('list')
    # 태그의 수만큼 반복한다.
    for result_tag in result_list : 
        data_dic['buryu'].append(re.sub(special_word, '', NoneXMLdata(result_tag.find('buryu'))))
        data_dic['sort_cd'].append(NoneXMLdata(result_tag.find('sort_cd')))
        data_dic['pum_cd'].append(NoneXMLdata(result_tag.find('pum_cd')))
        data_dic['pum_nm'].append(NoneXMLdata(result_tag.find('pum_nm')))
        data_dic['corp_cd_1'].append(NoneXMLdata(result_tag.find('corp_cd_1')))
        data_dic['corp_cd_2'].append(NoneXMLdata(result_tag.find('corp_cd_2')))
        data_dic['corp_cd_3'].append(NoneXMLdata(result_tag.find('corp_cd_3')))
        data_dic['corp_cd_4'].append(NoneXMLdata(result_tag.find('corp_cd_4')))
        data_dic['corp_cd_5'].append(NoneXMLdata(result_tag.find('corp_cd_5')))
        data_dic['corp_cd_6'].append(NoneXMLdata(result_tag.find('corp_cd_6')))
        data_dic['sum_tot'].append(NoneXMLdata(result_tag.find('sum_tot')))
        data_dic['date'].append(params['date'])

In [8]:
# 유통정보-가락시장 반입물량(정산후) 데이터
# pageidx=1&date=20161222
url = 'http://www.garak.co.kr/gongsa/jsp/gs/data_open/data.jsp'
params = {
     'id': ServiceKey  
    ,'pageidx': ''
    ,'date': ''
}

In [9]:
date_list = [d.strftime('%Y%m%d') for d in pd.date_range('20100101','20181231')]
for v in date_list:
    params['date'] = v
    params['pageidx'] = 1
    params_encode = "&".join("%s=%s" % (k,v) for k,v in params.items())
    data = connectionXML(url, params_encode)
    list_total_count = data.find('list_total_count').text
    total_pageidx = math.ceil(int(list_total_count) / 10)
    for i in range(1, total_pageidx + 1):
        params['pageidx'] = i
        params_encode = "&".join("%s=%s" % (k,v) for k,v in params.items())
        data = connectionXML(url, params_encode)
        getDataXML(data)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [11]:
df = pd.DataFrame.from_dict(data_dic)
df.to_csv('가락시장2010_2018.csv', index=False, encoding='utf-8-sig')
df.shape

(142543, 12)